In [4]:
def prepare_data_1():
    # Reading PIMMART data
    pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
        pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)
    
    # Reading Syndigo 259K data
    synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
        synd_ALL[i] = synd_ALL[i].astype(np.float64)
    
    # Stripping spaces from all columns
    df_obj = synd_ALL.select_dtypes(['object'])
    synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
    syndigo_mapped = synd_ALL
    pimmart = pim_gtin_mapped
    syndigo_mapped.drop_duplicates('GTIN', inplace = True)
    syndigo_mapped['ITEM_SUBCOM_text'] = \
    (syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()
    #syndigo_mapped['Level 1'].value_counts()
    print(f'syndigo_mapped.shape {syndigo_mapped.shape}') 
    print(f'syndigo_mapped.info() {syndigo_mapped.info()}') 
    print(f'syndigo_mapped.head() {syndigo_mapped.head()}') 
    print(f'syndigo_mapped.columns {syndigo_mapped.columns}') 
    
    print(f"syndigo_mapped['Level 1'].value_counts() {syndigo_mapped['Level 1'].value_counts(dropna = True)}")    
    return syndigo_mapped

In [5]:
def prepare_data_2(level__1):
    level_1 = level__1
    subset_df = syndigo_mapped[syndigo_mapped['Level 1'] == level_1]
    series = syndigo_mapped[syndigo_mapped['Level 1']==level__1]['Level 2'].value_counts(ascending=True)
    if series.tolist():
        subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series==1].index.tolist())].index)
    
    vect_subset = TfidfVectorizer(ngram_range = (1,2), max_features = 1000000)
    X_subset = vect_subset.fit_transform(subset_df.ITEM_SUBCOM_text) #scipy.sparse._csr.csr_matrix
    level2_id_map = dict(zip(subset_df['Level 2'].fillna('Other').unique(), range(subset_df['Level 2'].fillna('Other').nunique())))
    id2_level_map = dict(zip(range(subset_df['Level 2'].fillna('Other').nunique()), subset_df['Level 2'].fillna('Other').unique()))
    y_subset  = subset_df['Level 2'].fillna('Other').map(level2_id_map)
    
    return  X_subset, y_subset, subset_df

In [18]:
def proc_predict(level__1):
     level_1 = level__1
     subset_df = syndigo_mapped[syndigo_mapped['Level 1'] == level_1]
     series = syndigo_mapped[syndigo_mapped['Level 1']==level__1]['Level 2'].value_counts(ascending=True)   
     print(f'1. subset_df.shape {subset_df.shape} len(series) = { len(series)}\n series[:10] { series[:10]}')   
     if series.tolist():
         subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series==1].index.tolist())].index)
     print(f'2. subset_df.shape {subset_df.shape}')
           
     vect_subset = TfidfVectorizer(ngram_range = (1,2), max_features = 1000000)
     X_subset = vect_subset.fit_transform(subset_df.ITEM_SUBCOM_text)
     level2_id_map = dict(zip(subset_df['Level 2'].fillna('Other').unique(), range(subset_df['Level 2'].fillna('Other').nunique())))
     id2_level_map = dict(zip(range(subset_df['Level 2'].fillna('Other').nunique()), subset_df['Level 2'].fillna('Other').unique()))
     y_subset  = subset_df['Level 2'].fillna('Other').map(level2_id_map)
     A_train, A_test, B_train, B_test = train_test_split(subset_df.GTIN.tolist(), subset_df['Level 2'].tolist(), test_size=0.1, stratify=y_subset, random_state=42)
     X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size= 0.1, stratify=y_subset, random_state=42)
     lr_tf = LogisticRegressionCV(cv = 5, multi_class = 'multinomial', solver = 'saga', n_jobs=-1, max_iter=10000000)
     
     print(f"Training - \"{level_1}\"")
     lr_tf.fit(X_train, y_train)
     preds = lr_tf.predict(X_test)
     preds_lrtf = preds
     probs = lr_tf.predict_proba(X_test)
     preds_train = lr_tf.predict(X_train)
     probs_train = lr_tf.predict_proba(X_train)
     print(f"Done Training - \"{level_1}\"")
     return preds, preds_lrtf, probs, prebs_train, probs_train  

In [19]:
def proc_predict(filenamee):
    print("---------------------------------------------\FINAL COUNTS:\n---------------------------------------------")
    print("1)", len(A_test + A_train))
    print("2)", len(['Test']*len(A_test) + ['Train']*len(A_train)))
    print("3)", len(B_test + B_train))
    print("4)", len(testLevels + B_train))
    print("5)", len(testLevelss + trainLevelss))
    print("6)", int(len(test_new_proobs + train_new_proobs)))
    
    data = {
        'GTIN' : A_test + A_train,
        'Source': ['Test']*len(A_test) + ['Train']*len(A_train),
        'Actual Level 2' : B_test + B_train,
        'Actuals' : testLevels + B_train,
        'Predictions' : testLevelss + trainLevelss,
        'Scores' : test_new_proobs + train_new_proobs,
        'L1 Name' : level_1
    }
    
    df = pd.DataFrame(data)
    df = df.merge(pim_gtin_mapped[['GTIN', 'ITM_ID', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD',
        'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD',
        'SUBCOM_DSC', 'VND_ECOM_DSC']], on='GTIN', how='left')
    df = df[['ITM_ID', 'GTIN', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD',
        'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD', 'SUBCOM_DSC',
        'VND_ECOM_DSC', 'Source', 'Actual Level 2', 'Actuals', 'Predictions', 'Scores', 'L1 Name']]
    df.to_csv(DBFO + filenamee + '.csv', index=False)
    print(f"{filenamee}.csv   --file created successfully")
    return

In [20]:
preds, preds_lrtf, probs, prebs_train, probs_train = proc_predict(level__1)
proc_predict(filenamee)    

---------------------------------------------\FINAL COUNTS:
---------------------------------------------
1) 493
2) 493
3) 493
4) 493
5) 493
6) 493
Hospitality Supplies.csv   --file created successfully


TypeError: cannot unpack non-iterable NoneType object

In [1]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from pycaret.classification import setup
from pycaret.classification import compare_models
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix


In [2]:
import pandas as pd, numpy as np
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
DBFS = path
DBFO = path
DBFM = path
DBFR = path


In [6]:
syndigo_mapped = prepare_data_1()

syndigo_mapped.shape (259085, 24)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 259085 entries, 0 to 259084
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   GTIN              259085 non-null  float64
 1   Level 1           257953 non-null  object 
 2   Level 2           256935 non-null  object 
 3   Level 3           247350 non-null  object 
 4   Level 4           179389 non-null  object 
 5   Level 5           45755 non-null   object 
 6   Level 6           5286 non-null    object 
 7   Level 7           1542 non-null    object 
 8   Level 8           179 non-null     object 
 9   Level 9           2 non-null       object 
 10  Level 10          0 non-null       object 
 11  ITM_ID            259069 non-null  float64
 12  PMY_DPT_CD        258915 non-null  float64
 13  PMY_DPT_DSC       258915 non-null  object 
 14  REC_DPT_CD        258915 non-null  object 
 15  REC_DPT_DSC       258915 non-null 

In [7]:

vect = TfidfVectorizer(ngram_range= (1,2), max_features = 100000)
X_subset, y_subset, subset_df =  prepare_data_2("Toys / Games / Hobbies")

In [8]:
y_subset = y_subset.reset_index().drop('index', axis = 1)


In [10]:
X_train, X_validation, Y_train, Y_validation = train_test_splity(X_subset, y_subset, test_size=0.10, random_state=1)

NameError: name 'train_test_splity' is not defined

In [66]:
df1=dfx.reset_index().drop('index', axis = 1) 
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,16564,16565,16566,16567,16568,16569,16570,16571,16572,16573
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
df2=dfy.reset_index().drop('index', axis = 1)

df2.head()

,Level 2
0,Games & Puzzles
1,Toy Figures
2,Toy Figures
3,Games & Puzzles
4,Toy Figures


In [67]:
df3 = pd.merge(df1, df2, left_index=True, right_index=True)
df3.head()

,0,1,2,3,4,5,6,7,8,9,...,16565,16566,16567,16568,16569,16570,16571,16572,16573,Level 2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Games & Puzzles
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Figures
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Figures
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Games & Puzzles
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Toy Figures


In [68]:
df3['Level 2'].value_counts(dropna = False)

Toy Figures                    1139
Games & Puzzles                 540
Construction & Modeling         394
Baby Toys & Games               372
Toy Vehicles                    258
Outdoor Toys & Games            191
Role Playing & Pretend Play     164
Sports Toys & Games              82
Remote & Radio Control           38
Nature / Science                 12
Name: Level 2, dtype: int64

In [61]:
dfz['Level 2'].value_counts(dropna = True)

Toy Figures                    22
Games & Puzzles                13
Construction & Modeling         6
Baby Toys & Games               5
Role Playing & Pretend Play     4
Toy Vehicles                    2
Remote & Radio Control          1
Outdoor Toys & Games            1
Name: Level 2, dtype: int64

In [ ]:
subset_df.

In [56]:
subset_df.head(10)

,GTIN,Level 1,Level 2,Level 3,Level 4,Level 5,Level 6,Level 7,Level 8,Level 9,...,REC_DPT_CD,REC_DPT_DSC,DPT_CD,DPT_DSC,COM_CD,COM_DSC,SUBCOM_CD,SUBCOM_DSC,VND_ECOM_DSC,ITEM_SUBCOM_text
53,6.305095e+11,Toys / Games / Hobbies,Games & Puzzles,Board Games,Family,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,842.0,GAMES,87608.0,BOARD GAMES,Monopoly® Board Game,monopoly® board game board games
71,6.305095e+11,Toys / Games / Hobbies,Toy Figures,Action Figures,Individual,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,973.0,BASIC GIRL,87662.0,FASHION DOLLS,DPR TANGLED STORY FIG,dpr tangled story fig fashion dolls
80,8.879616e+11,Toys / Games / Hobbies,Toy Figures,Action Figures,Individual,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,954.0,BASIC PRE-SCHOOL,87651.0,FISHER PRICE,Fisher-Price Imaginext Jurassic World Dinosaur...,fisher-price imaginext jurassic world dinosaur...
108,6.894230e+11,Toys / Games / Hobbies,Games & Puzzles,Spinning Tops,NaN,NaN,NaN,NaN,NaN,NaN,...,01,GROCERY,1.0,GROC-ALL OTHER,135.0,KOSHER FOODS AND PRODUCTS,18278.0,KOSHER CANDY,GEDILLA CANDY - CHANUKAH DREIDLE,gedilla candy - chanukah dreidle kosher candy
170,8.879616e+11,Toys / Games / Hobbies,Toy Figures,Action Figures,Sets,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,973.0,BASIC GIRL,97305.0,BARBIE DOLLS,MTL BRB LIC FASHION LOOKS,mtl brb lic fashion looks barbie dolls
180,6.305096e+11,Toys / Games / Hobbies,Toy Figures,Action Figures,Individual,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,834.0,BASIC BOYS TOYS,87605.0,ACTION FIGURES,SPD 6 QUICK SHOT,spd 6 quick shot action figures
205,6.535700e+11,Toys / Games / Hobbies,Games & Puzzles,Puzzles,Brain Teasers,Assembly / Disassembly,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,842.0,GAMES,87614.0,CARD GAMES,Hasbro Gaming Rubik's Cube Game,hasbro gaming rubik's cube game card games
290,6.305096e+11,Toys / Games / Hobbies,Toy Figures,Stuffed Animals,NaN,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,954.0,BASIC PRE-SCHOOL,87646.0,PRESCHOOL TOYS,Hasbro Sesame Street Abby Cadabby Mini Plush Doll,hasbro sesame street abby cadabby mini plush d...
297,8.879616e+11,Toys / Games / Hobbies,Toy Figures,Action Figures,Sets,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,834.0,BASIC BOYS TOYS,87668.0,CONSTRUCTION,MEGA Mega Construx Pokmon Blastoise,mega mega construx pokmon blastoise construction
361,6.535696e+11,Toys / Games / Hobbies,Toy Figures,Action Figures,Individual,NaN,NaN,NaN,NaN,NaN,...,87,GM,64.0,TOYS,834.0,BASIC BOYS TOYS,87605.0,ACTION FIGURES,TRA3 CYBERTECH DLX,tra3 cybertech dlx action figures


In [53]:
subset_df['Level 2']

53                Games & Puzzles
71                    Toy Figures
80                    Toy Figures
108               Games & Puzzles
170                   Toy Figures
                   ...           
258215    Construction & Modeling
258376               Toy Vehicles
258717                Toy Figures
258827                Toy Figures
258831    Construction & Modeling
Name: Level 2, Length: 3190, dtype: object

In [54]:
subset_df[['Level 2']]

,Level 2
53,Games & Puzzles
71,Toy Figures
80,Toy Figures
108,Games & Puzzles
170,Toy Figures
...,...
258215,Construction & Modeling
258376,Toy Vehicles
258717,Toy Figures
258827,Toy Figures


In [55]:
subset_df[['Level 2']].index

Int64Index([    53,     71,     80,    108,    170,    180,    205,    290,
               297,    361,
            ...
            257772, 257788, 257916, 257958, 257985, 258215, 258376, 258717,
            258827, 258831],
           dtype='int64', length=3190)

In [11]:
# Load libraries
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


In [15]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X_subset, X_subset, test_size=0.10, random_state=1)
models = []
models.append(('LRC', LogisticRegressionCV(solver='saga', multi_class='multinomial')))
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
#?models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
#?models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_subset, y_subset, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))


LRC: 0.771473 (0.007986)
LR: 0.731975 (0.022529)
KNN: 0.731661 (0.011912)
CART: 0.736364 (0.020673)
SVM: 0.357053 (0.000940)


In [16]:
path = 'C:\\users\\iny2819\\kroger\\Data\\'
syndigo_mapped.to_excel(path + "syndigo_mapped.xlsx",index=None)

In [3]:
# Reading PIMMART data
pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
    pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)

# Reading Syndigo 259K data
synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
    synd_ALL[i] = synd_ALL[i].astype(np.float64)

# Stripping spaces from all columns
df_obj = synd_ALL.select_dtypes(['object'])
synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [4]:
syndigo_mapped = synd_ALL
pimmart = pim_gtin_mapped

In [5]:
syndigo_mapped.drop_duplicates('GTIN', inplace = True)

In [6]:
syndigo_mapped['ITEM_SUBCOM_text'] = \
(syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()

In [7]:
syndigo_mapped['Level 1'].value_counts()

Food / Beverages                            131593
Health & Beauty                              62259
Beer / Wine / Spirits                        17992
Cleaning & Janitorial                         9084
Livestock & Pet Supplies                      6325
Kitchen & Bathroom                            5268
Home & Venue Decoration                       3924
Toys / Games / Hobbies                        3191
Gardening & Outdoors                          2208
Childcare                                     2111
Office Supplies                               1671
Electronics                                   1499
Apparel                                       1283
Lighting & Fans                               1252
Tobacco Products                              1252
Not classified                                1018
Hardware                                       923
Arts & Crafts                                  803
Automotive                                     778
Appliances                     

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(ngram_range= (1,2), max_features = 800000)

In [9]:
syndigo_mapped['Level 1'].value_counts()

Food / Beverages                            131593
Health & Beauty                              62259
Beer / Wine / Spirits                        17992
Cleaning & Janitorial                         9084
Livestock & Pet Supplies                      6325
Kitchen & Bathroom                            5268
Home & Venue Decoration                       3924
Toys / Games / Hobbies                        3191
Gardening & Outdoors                          2208
Childcare                                     2111
Office Supplies                               1671
Electronics                                   1499
Apparel                                       1283
Lighting & Fans                               1252
Tobacco Products                              1252
Not classified                                1018
Hardware                                       923
Arts & Crafts                                  803
Automotive                                     778
Appliances                     

## RUN UNTIL HERE only ONCE

In [10]:
#level__1 = "Toys / Games / Hobbies"
#filenamee = "TOYS_GAMES_HOBBIES"

# level__1 = "Tobacco Products"
# filenamee = "TOBACCO_PRODUCTS"

# level__1 = "Cleaning & Janitorial"
# filenamee = "CLEANING_JANITORIAL"

# level__1 = "Office Supplies"
# filenamee = "OFFICE_SUPPLIES"

# level__1 = "Beer / Wine / Spirits"
# filenamee = "BEER_WINE_SPIRITS"

# level__1 = "Livestock & Pet Supplies"
# filenamee = "LIVESTOCK_PET_SUPPLIES"

# level__1 = "Kitchen & Bathroom"
# filenamee = "KITCHEN_BATHROOM"

level__1 = "Hospitality Supplies"
filenamee = "HOSPITALITY_SUPPLIES"

# level__1 = "Home & Venue Decoration"
# filenamee = "HOME_VENUE_DECORATION"

# level__1 = "Heating / Ventilation / Air Conditioning"
# filenamee = "HEATING_VENTILATION_AIR_CONDITIONING"

# level__1 = "Gardening & Outdoors"
# filenamee = "GARDENING_OUTDOORS"

# level__1 = "Sports & Outdoor Recreation Equipment"
# filenamee = "SPORTS_OUTDOOR_RECREATION_EQUIPMENT"

syndigo_mapped[syndigo_mapped['Level 1']==level__1]['Level 2'].value_counts(ascending=True)

Note Pads & Paper                          1
Beverage Carriers                          1
Beverage Container Covers                  1
Dispensers                                 3
Comment Cards                              3
Ice Buckets                                4
Food & Beverage Condiments                 5
Paper / Plastic / Foam Cups Sleeves        5
Plastic Flatware                          65
Paper / Plastic / Foam Cups              138
Paper / Plastic / Foam Plates & Bowls    270
Name: Level 2, dtype: int64

In [11]:
level_1 = level__1
subset_df = syndigo_mapped[syndigo_mapped['Level 1'] == level_1]
series = syndigo_mapped[syndigo_mapped['Level 1']==level__1]['Level 2'].value_counts(ascending=True)
if series.tolist():
    subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series==1].index.tolist())].index)

vect_subset = TfidfVectorizer(ngram_range = (1,2), max_features = 1000000)
X_subset = vect_subset.fit_transform(subset_df.ITEM_SUBCOM_text)
level2_id_map = dict(zip(subset_df['Level 2'].fillna('Other').unique(), range(subset_df['Level 2'].fillna('Other').nunique())))
id2_level_map = dict(zip(range(subset_df['Level 2'].fillna('Other').nunique()), subset_df['Level 2'].fillna('Other').unique()))
y_subset  = subset_df['Level 2'].fillna('Other').map(level2_id_map)
A_train, A_test, B_train, B_test = train_test_split(subset_df.GTIN.tolist(), subset_df['Level 2'].tolist(), test_size=0.1, stratify=y_subset, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size= 0.1, stratify=y_subset, random_state=42)
lr_tf = LogisticRegressionCV(cv = 5, multi_class = 'multinomial', solver = 'saga', n_jobs=-1, max_iter=10000000)

print(f"Training - \"{level_1}\"")
lr_tf.fit(X_train, y_train)
preds = lr_tf.predict(X_test)
preds_lrtf = preds
probs = lr_tf.predict_proba(X_test)
preds_train = lr_tf.predict(X_train)
probs_train = lr_tf.predict_proba(X_train)
print(f"Done Training - \"{level_1}\"")

Training - "Hospitality Supplies"


C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Done Training - "Hospitality Supplies"


In [12]:
level_num = []
testProobs = []
number = 1

print(len(preds))
for i in range(len(preds)):
    for j in range(number):
        try:
            level_num.append(int(np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][j])[0]))
        except TypeError:
            level_num.append(int(np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][2])[0][0]))
            print("problem occurred with: ", i, j, flush=True)
    testProobs.append(probs[i][np.argsort(probs[i])][::-1][:1])

test_new_proobs = []
for i in range(len(testProobs)):
    test_new_proobs.append(testProobs[i].tolist())
test_new_proobs = [element for sublist in list(test_new_proobs) for element in sublist]

50


In [13]:
level_num = []
trainProobs = []

print(len(preds_train))
for i in range(len(preds_train)):
    for j in range(number):
        try:
            level_num.append(int(np.where(probs_train[i]==probs_train[i][np.argsort(probs_train[i])][::-1][:3][j])[0]))
        except TypeError:
            level_num.append(int(np.where(probs_train[i]==probs_train[i][np.argsort(probs_train[i])][::-1][:3][2])[0][0]))
            print("problem occurred with: ", i, j, flush=True)
    trainProobs.append(probs_train[i][np.argsort(probs_train[i])][::-1][:1]) 

train_new_proobs = []
for i in range(len(trainProobs)):
    train_new_proobs.append(trainProobs[i].tolist())
train_new_proobs = [element for sublist in list(train_new_proobs) for element in sublist]

443


In [14]:
testLevels = []
for j in y_test:
    testLevels.append([i for i in level2_id_map if level2_id_map[i]==j][0])

testLevelss = []
for j in preds:
    testLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])

trainLevelss = []
for j in preds_train:
    trainLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])

In [15]:
print("---------------------------------------------\FINAL COUNTS:\n---------------------------------------------")
print("1)", len(A_test + A_train))
print("2)", len(['Test']*len(A_test) + ['Train']*len(A_train)))
print("3)", len(B_test + B_train))
print("4)", len(testLevels + B_train))
print("5)", len(testLevelss + trainLevelss))
print("6)", int(len(test_new_proobs + train_new_proobs)))

---------------------------------------------\FINAL COUNTS:
---------------------------------------------
1) 493
2) 493
3) 493
4) 493
5) 493
6) 493


In [16]:
data = {
    'GTIN' : A_test + A_train,
    'Source': ['Test']*len(A_test) + ['Train']*len(A_train),
    'Actual Level 2' : B_test + B_train,
    'Actuals' : testLevels + B_train,
    'Predictions' : testLevelss + trainLevelss,
    'Scores' : test_new_proobs + train_new_proobs,
    'L1 Name' : level_1
}

df = pd.DataFrame(data)
df = df.merge(pim_gtin_mapped[['GTIN', 'ITM_ID', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD',
    'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD',
    'SUBCOM_DSC', 'VND_ECOM_DSC']], on='GTIN', how='left')
df = df[['ITM_ID', 'GTIN', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD',
    'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD', 'SUBCOM_DSC',
    'VND_ECOM_DSC', 'Source', 'Actual Level 2', 'Actuals', 'Predictions', 'Scores', 'L1 Name']]

## Keep the output of the below cell for all Level 1 executed

In [17]:
df.to_csv(DBFO + filenamee + '.csv', index=False)
print(f"{filenamee}.csv   --file created successfully")

HOSPITALITY_SUPPLIES.csv   --file created successfully
